# RNSA Breast Cancer Detector:

- ¿Que vamos a utlizar?

- Librerias

In [99]:
# Librerias

import pandas as pd


- Carga de datos

In [117]:
#Vamos a crear un bloque de comprobación para asegurar que los datos se cargan sin problema.

def GetData():
    print("Loading data...\n==========\n")
    try:
        data_train = pd.read_csv("../data/raw/train.csv")
        print("· Train data loaded")
    except Exception as e:
        print(f"· Train data is not loaded: Exception {e}\n==========\n")
    
    try:
        data_test = pd.read_csv("../data/raw/test.csv")
        print("· Train data is loaded")
    except Exception as e:
        print(f"· Test data is not loaded: Exception {e}\n==========\n")

    try:
        data_submision = pd.read_csv("../data/raw/sample_submission.csv")
        print("· Submission data is loaded\n==========\n")
    
    except Exception as e:
        print(f"· Submission data is not loaded: Exception {e}\n==========\n")

    return data_train, data_test, data_submision

#Desempaquetamos los Datasets si se han cargado correctamente.
try:
    dataTrain, dataTest, dataSub = GetData()
    print("...Data loaded succesfully")
except Exception as e:
    print(f"Error: {e}")

Loading data...

· Train data loaded
· Train data is loaded
· Submission data is loaded

...Data loaded succesfully


- Una vez cargado los datos vamos a analizar los Datasets.

In [118]:
def DataAnalysis(dataset):
    print(f"Data Dimensions: {dataset.shape}\n")
    
    try:
        nan_filter = dataset.isna().sum()
        data_nan = dataset[nan_filter > 0]

        for col, n in data_nan.items():
            print(f"Column {col}: {n} NaN values")
    except:
        print("Non NaN values in Dataset")

DataAnalysis(dataTrain)

Data Dimensions: (54706, 14)

Non NaN values in Dataset


/tmp/ipykernel_516/3923148241.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data_nan = dataset[nan_filter > 0]


In [49]:
dataTest.head()

,site_id,patient_id,image_id,laterality,view,age,implant,machine_id,prediction_id
0,2,10008,736471439,L,MLO,81,0,21,10008_L
1,2,10008,1591370361,L,CC,81,0,21,10008_L
2,2,10008,68070693,R,MLO,81,0,21,10008_R
3,2,10008,361203119,R,CC,81,0,21,10008_R


In [50]:
dataSub.head()

,prediction_id,cancer
0,10008_L,0.021168
1,10008_R,0.021168
